# Import packages

In [111]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import shap
from pathlib import Path

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix
from scipy.stats import shapiro
from matplotlib import rcParams
from cycler import cycler
from sklearn.inspection import permutation_importance

rcParams.update({'figure.autolayout': True})

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

Path('../results').mkdir(exist_ok=True)
linestyle_cycler =  [
     'solid','dotted', 'dashed','dashdot',
     (0, (1, 1)),(0, (1, 1))]
plt.rc('axes', prop_cycle=(cycler('color', ['r', 'g', 'b', 'y', 'c', 'k']) + cycler('linestyle', linestyle_cycler)))

## Load lipid data

In [112]:
lipid_data_df = pd.read_csv(f'all_data.csv')

In [113]:
lipid_data_df

,Date,Blood Type,Person,Cer d18:2_16:0,Cer d18:1_18:0,Cer d18:1_18:1,Cer d18:1_16:0,Cer d18:1_20:0,Cer d18:1_22:0,Cer d18:1_24:0,Cer d18:1_24:1,Cer d40:2,Cer d41:1,Cer d41:2,Cer d43:1,Cer d44:1,Cer d44:2,Hex2Cer 32:1;2O,Hex2Cer 34:1;2O,Hex2Cer 34:2;2O,Hex2Cer 42:2;2O,Hex2Cer 42:3;2O,Hex3Cer 34:1;2O,HexCer 14:2;2O/28:0,HexCer 41:1;2O,HexCer 42:1;2O,LPA 17:0,LPC 14:0 SN2,LPC 14:0 SN1,LPC 15:0 SN2,LPC 15:0 SN1,LPC 16:0 SN2,LPC 16:0 SN1,LPC 16:1 SN1,LPC 16:1 SN2,LPC 17:0 SN2,LPC 17:0 SN1,LPC 18:0 SN1,LPC 18:1 SN1,LPC 18:1 SN2,LPC 18:2 SN1,LPC 18:2 SN2,LPC 18:3,LPC 19:0 SN2,LPC 20:0 SN2,LPC 20:1 SN1,LPC 20:2 SN2,LPC 20:3 SN1,LPC 20:4 SN2,LPC 20:4 SN1,LPC 20:5 SN2,LPC 20:5 SN1,LPC 22:0,LPC 22:5 SN1,LPC 22:6 SN2,LPC 24:0,LPC O-18:0,LPC O-18:1,LPC O-20:0,LPC O-22:0,LPC O-24:1,LPE 16:0 SN1,LPE 16:0 SN2,LPE 18:0,LPE 18:0 SN2,LPE 18:1 SN2,LPE 18:1 SN1,LPE 18:2 SN2,LPE 18:2 SN1,LPE 20:4 SN2,LPE 20:4 SN1,LPE 20:5,LPE 22:4,LPE 22:6,LPE O-16:1,LPE O-18:1,LPI 18:0,LPI 20:4,PA 16:0_18:1,PA 16:0_18:2,PC 12:0_16:0,PC 14:0_16:1,PC 14:0_18:2,PC 14:0_20:4,PC 14:0_22:6,PC 15:0_16:0,PC 15:0_18:2,PC 15:0_20:3,PC 15:0_20:4,PC 16:0_14:0,PC 16:0_16:0,PC 16:0_17:0,PC 16:0_18:1,PC 16:0_18:2,PC 16:0_18:3,PC 16:0_20:1,PC 16:0_22:6,PC 16:1_20:4,PC 16:1_22:6,PC 17:0_18:1,PC 17:0_18:2,PC 17:0_20:4,PC 17:0_22:6,PC 17:1_20:4,PC 18:0_18:1,PC 18:0_18:2,PC 18:0_20:1,PC 18:0_20:4,PC 18:0_22:4,PC 18:0_22:6,PC 18:0_24:4,PC 18:1_14:0,PC 18:1_18:2,PC 18:1_20:4,PC 18:1_22:6,PC 18:2_18:2 Cis,PC 18:2_20:4,PC 18:3_20:4,PC 19:0_18:1,PC 19:0_18:2,PC 20:1_18:1,PC 20:1_18:2,PC 20:1_20:4,PC 20:2_20:4,PC 20:4_20:4 Cis,PC 22:6_20:4,PC O-16:0_16:0,PC O-16:0_20:3,PC O-16:0_20:4,PC O-16:0_20:5,PC O-16:1_16:0,PC O-16:1_18:2,PC O-16:1_20:4,PC O-16:1_22:6,PC O-18:0_18:1,PC O-18:0_18:2,PC O-18:0_20:4,PC O-18:0_22:6,PC O-18:1_16:0,PC O-18:1_20:4,PC O-18:1_20:5,PC O-18:1_22:6,PC O-18:2_22:4,PC O-20:0_20:4,PC O-20:1_20:4,PC O-22:0_20:4,PC O-22:1_20:4,PC O-22:1_22:6,PC O-22:2_20:4,PC O-24:1_20:4,PE 16:0_16:0,PE 16:0_16:1,PE 16:0_18:1,PE 16:0_18:2,PE 16:0_20:4,PE 16:0_20:5,PE 16:0_22:6,PE 18:0_18:1,PE 18:0_20:4,PE 18:0_22:6,PE 18:1_18:1 Cis,PE 18:1_18:2,PE 18:2_18:2,PE 18:2_20:4,PE 18:3_18:3,PE O-16:1_20:4,PE O-16:1_22:6,PE O-17:1_20:4,PE O-17:1_22:6,PE O-18:1_18:1,PE O-18:1_20:4,PE O-18:2_22:6,PE O-20:0_18:2,PE O-20:1_18:2,PE O-20:1_20:4,PG 16:0_18:2,PG 18:0_20:4,PG 18:1_18:1 Cis,PI 16:0_18:1,PI 16:0_18:2,PI 16:0_20:4,PI 16:0_22:5,PI 18:0_18:2,PI 18:0_20:3,PI 18:0_20:4,PI 18:0_22:4,PI 18:1_18:2,PS 16:0_16:0,PS 16:0_18:1,PS 18:0_18:0,PS 18:0_18:1,PS 18:0_18:2,PS 18:0_20:4,PS 18:0_22:6,SM 34:0;3O,SM 34:1;3O,SM 34:2;3O,SM d18:1_16:0,SM d18:1_18:0,SM d18:1_18:1,SM d18:1_24:0,SM d18:1_24:1,SM d24:2_14:0,SM d32:0,SM d32:1,SM d32:2,SM d33:1,SM d33:2,SM d34:0,SM d34:2,SM d35:1,SM d35:2,SM d36:0,SM d36:3,SM d38:1,SM d38:3,SM d39:1,SM d40:1,SM d40:2,SM d40:3,SM d41:1,SM d41:2,SM d41:3,SM d44:2,SM d44:3,Cer d18:1_17:0,Cer d43:2,HexCer 18:1;2O/16:0,LPC 17:0,LPC 19:0 SN1,LPC 20:0 SN1,LPC 20:2 SN1,LPC 20:5,LPC 22:4,LPC O-16:0,PA 16:0_20:4,PC 15:0_18:1,PC 16:0_20:3,PC 16:0_20:4,PC 16:0_20:5,PC 16:0_22:4,PC 16:0_22:5,PC 18:0_20:2,PC 18:0_20:3,PC 18:1_18:1 Cis,PC 18:1_22:0,PC 20:0_20:0,PC 22:0_18:2,PC O-16:0_16:1,PC O-16:0_18:1,PC O-20:0_22:6,PC O-20:1_22:4,PC O-20:2_20:4,PC O-22:0_18:2,PC O-22:2_18:2,PE O-16:1_18:1,PE O-16:1_22:4,PS 16:0_20:4,PS 18:2_18:2,SM d38:2,LPC 18:0 SN2,PC 16:0_16:1,PC 18:1_22:4,PE O-18:1_20:3,PE O-18:1_22:6,PE O-20:0_22:6,PG 16:0_18:1,PI 16:0_16:1,PI 18:0_18:1,PI 18:1_20:4,LPC 20:3 SN2,LPC 22:6 SN1,LPC O-22:1,PC 18:0_16:0,PC 18:0_22:5,PC 19:0_20:4,PC O-18:0_22:4,PC O-18:2_20:4,PC O-24:2_20:4,PE O-18:3_20:4,PI 18:0_22:5
0,210625,Plasma,1,0.05,0.27,0.08,0.52,0.82,3.37,9.62,0.88,0.84,4.21,0.84,1.15,0.21,0.08,0.19,1.49,0.00,0.26,0.06,0.17,0.70,0.88,2.22,0.04,0.37,8.37,0.78,4.23,34.14,404.07,5.52,1.03,0.73,5.58,175.02,124.43,11.10,158.38,17.16,1.10,0.64,0.79,1.21,0.78,2.88,5.60,28.41,0.77,2.65,0.29,1.21,2.45,0.55,1.01,1.27,0.27,0.35,0.49,2.09,0.30,2.47,0.30,0.21,2.82,0.91,5.61,1.34,3.91,0

In [114]:
lipid_data_df.isna().any().value_counts()

False    158
True     123
dtype: int64

In [115]:
print(lipid_data_df.isna().any().sort_values(ascending=False))

PI 18:0_22:5            True
PA 16:0_18:1            True
PC 17:1_20:4            True
SM d38:3                True
PC 16:1_22:6            True
PC 16:1_20:4            True
PC 16:0_20:1            True
PC 16:0_18:2            True
PC 16:0_18:1            True
Cer d18:1_17:0          True
HexCer 18:1;2O/16:0     True
LPC 17:0                True
PC 14:0_16:1            True
LPC 19:0 SN1            True
LPC 20:0 SN1            True
LPI 20:4                True
PC 16:0_22:4            True
LPI 18:0                True
LPC 20:2 SN1            True
LPE O-16:1              True
LPC 20:5                True
LPC 22:4                True
LPE 20:5                True
LPC O-16:0              True
PA 16:0_20:4            True
PC 15:0_18:1            True
PC 16:0_20:3            True
PC 16:0_20:4            True
LPE 18:1 SN2            True
LPE 18:0 SN2            True
PC 18:0_18:2            True
PC 18:0_20:4            True
PC 18:0_24:4            True
PC 18:1_14:0            True
PE O-20:1_18:2

In [116]:
lipid_data_df.drop(columns=['Person', 'Date']).describe()

,Cer d18:2_16:0,Cer d18:1_18:0,Cer d18:1_18:1,Cer d18:1_16:0,Cer d18:1_20:0,Cer d18:1_22:0,Cer d18:1_24:0,Cer d18:1_24:1,Cer d40:2,Cer d41:1,Cer d41:2,Cer d43:1,Cer d44:1,Cer d44:2,Hex2Cer 32:1;2O,Hex2Cer 34:1;2O,Hex2Cer 34:2;2O,Hex2Cer 42:2;2O,Hex2Cer 42:3;2O,Hex3Cer 34:1;2O,HexCer 14:2;2O/28:0,HexCer 41:1;2O,HexCer 42:1;2O,LPA 17:0,LPC 14:0 SN2,LPC 14:0 SN1,LPC 15:0 SN2,LPC 15:0 SN1,LPC 16:0 SN2,LPC 16:0 SN1,LPC 16:1 SN1,LPC 16:1 SN2,LPC 17:0 SN2,LPC 17:0 SN1,LPC 18:0 SN1,LPC 18:1 SN1,LPC 18:1 SN2,LPC 18:2 SN1,LPC 18:2 SN2,LPC 18:3,LPC 19:0 SN2,LPC 20:0 SN2,LPC 20:1 SN1,LPC 20:2 SN2,LPC 20:3 SN1,LPC 20:4 SN2,LPC 20:4 SN1,LPC 20:5 SN2,LPC 20:5 SN1,LPC 22:0,LPC 22:5 SN1,LPC 22:6 SN2,LPC 24:0,LPC O-18:0,LPC O-18:1,LPC O-20:0,LPC O-22:0,LPC O-24:1,LPE 16:0 SN1,LPE 16:0 SN2,LPE 18:0,LPE 18:0 SN2,LPE 18:1 SN2,LPE 18:1 SN1,LPE 18:2 SN2,LPE 18:2 SN1,LPE 20:4 SN2,LPE 20:4 SN1,LPE 20:5,LPE 22:4,LPE 22:6,LPE O-16:1,LPE O-18:1,LPI 18:0,LPI 20:4,PA 16:0_18:1,PA 16:0_18:2,PC 12:0_16:0,PC 14:0_16:1,PC 14:0_18:2,PC 14:0_20:4,PC 14:0_22:6,PC 15:0_16:0,PC 15:0_18:2,PC 15:0_20:3,PC 15:0_20:4,PC 16:0_14:0,PC 16:0_16:0,PC 16:0_17:0,PC 16:0_18:1,PC 16:0_18:2,PC 16:0_18:3,PC 16:0_20:1,PC 16:0_22:6,PC 16:1_20:4,PC 16:1_22:6,PC 17:0_18:1,PC 17:0_18:2,PC 17:0_20:4,PC 17:0_22:6,PC 17:1_20:4,PC 18:0_18:1,PC 18:0_18:2,PC 18:0_20:1,PC 18:0_20:4,PC 18:0_22:4,PC 18:0_22:6,PC 18:0_24:4,PC 18:1_14:0,PC 18:1_18:2,PC 18:1_20:4,PC 18:1_22:6,PC 18:2_18:2 Cis,PC 18:2_20:4,PC 18:3_20:4,PC 19:0_18:1,PC 19:0_18:2,PC 20:1_18:1,PC 20:1_18:2,PC 20:1_20:4,PC 20:2_20:4,PC 20:4_20:4 Cis,PC 22:6_20:4,PC O-16:0_16:0,PC O-16:0_20:3,PC O-16:0_20:4,PC O-16:0_20:5,PC O-16:1_16:0,PC O-16:1_18:2,PC O-16:1_20:4,PC O-16:1_22:6,PC O-18:0_18:1,PC O-18:0_18:2,PC O-18:0_20:4,PC O-18:0_22:6,PC O-18:1_16:0,PC O-18:1_20:4,PC O-18:1_20:5,PC O-18:1_22:6,PC O-18:2_22:4,PC O-20:0_20:4,PC O-20:1_20:4,PC O-22:0_20:4,PC O-22:1_20:4,PC O-22:1_22:6,PC O-22:2_20:4,PC O-24:1_20:4,PE 16:0_16:0,PE 16:0_16:1,PE 16:0_18:1,PE 16:0_18:2,PE 16:0_20:4,PE 16:0_20:5,PE 16:0_22:6,PE 18:0_18:1,PE 18:0_20:4,PE 18:0_22:6,PE 18:1_18:1 Cis,PE 18:1_18:2,PE 18:2_18:2,PE 18:2_20:4,PE 18:3_18:3,PE O-16:1_20:4,PE O-16:1_22:6,PE O-17:1_20:4,PE O-17:1_22:6,PE O-18:1_18:1,PE O-18:1_20:4,PE O-18:2_22:6,PE O-20:0_18:2,PE O-20:1_18:2,PE O-20:1_20:4,PG 16:0_18:2,PG 18:0_20:4,PG 18:1_18:1 Cis,PI 16:0_18:1,PI 16:0_18:2,PI 16:0_20:4,PI 16:0_22:5,PI 18:0_18:2,PI 18:0_20:3,PI 18:0_20:4,PI 18:0_22:4,PI 18:1_18:2,PS 16:0_16:0,PS 16:0_18:1,PS 18:0_18:0,PS 18:0_18:1,PS 18:0_18:2,PS 18:0_20:4,PS 18:0_22:6,SM 34:0;3O,SM 34:1;3O,SM 34:2;3O,SM d18:1_16:0,SM d18:1_18:0,SM d18:1_18:1,SM d18:1_24:0,SM d18:1_24:1,SM d24:2_14:0,SM d32:0,SM d32:1,SM d32:2,SM d33:1,SM d33:2,SM d34:0,SM d34:2,SM d35:1,SM d35:2,SM d36:0,SM d36:3,SM d38:1,SM d38:3,SM d39:1,SM d40:1,SM d40:2,SM d40:3,SM d41:1,SM d41:2,SM d41:3,SM d44:2,SM d44:3,Cer d18:1_17:0,Cer d43:2,HexCer 18:1;2O/16:0,LPC 17:0,LPC 19:0 SN1,LPC 20:0 SN1,LPC 20:2 SN1,LPC 20:5,LPC 22:4,LPC O-16:0,PA 16:0_20:4,PC 15:0_18:1,PC 16:0_20:3,PC 16:0_20:4,PC 16:0_20:5,PC 16:0_22:4,PC 16:0_22:5,PC 18:0_20:2,PC 18:0_20:3,PC 18:1_18:1 Cis,PC 18:1_22:0,PC 20:0_20:0,PC 22:0_18:2,PC O-16:0_16:1,PC O-16:0_18:1,PC O-20:0_22:6,PC O-20:1_22:4,PC O-20:2_20:4,PC O-22:0_18:2,PC O-22:2_18:2,PE O-16:1_18:1,PE O-16:1_22:4,PS 16:0_20:4,PS 18:2_18:2,SM d38:2,LPC 18:0 SN2,PC 16:0_16:1,PC 18:1_22:4,PE O-18:1_20:3,PE O-18:1_22:6,PE O-20:0_22:6,PG 16:0_18:1,PI 16:0_16:1,PI 18:0_18:1,PI 18:1_20:4,LPC 20:3 SN2,LPC 22:6 SN1,LPC O-22:1,PC 18:0_16:0,PC 18:0_22:5,PC 19:0_20:4,PC O-18:0_22:4,PC O-18:2_20:4,PC O-24:2_20:4,PE O-18:3_20:4,PI 18:0_22:5
count,60.000000,30.000000,30.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,45.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,45.000000,30.000000,60.000000,15.000000,60.000000,60.000000,60.000000,60.000000,45.000000,15.000000,45.000000,60.000000,60.000000,60.000000,60.000000,45.00000

# Remove lipids that does not appear in every person or type

In [117]:
lipid_data_df_without_na = lipid_data_df.dropna(axis='columns')
plasma = lipid_data_df_without_na[lipid_data_df_without_na['Blood Type'] == 'Plasma']
serum = lipid_data_df_without_na[lipid_data_df_without_na['Blood Type'] == 'Serum']
blood_edta = lipid_data_df_without_na[lipid_data_df_without_na['Blood Type'] == 'Blood EDTA - Serum']
dbs_venous = lipid_data_df_without_na[lipid_data_df_without_na['Blood Type'] == 'DBS - Venous']
dbs_finger = lipid_data_df_without_na[lipid_data_df_without_na['Blood Type'] == 'DBS - Finger']
blood_type_dict = {'Plasma': plasma, 
              'Serum':serum,
              'Blood EDTA - Serum': blood_edta,
              'DBS - Venous': dbs_venous,
              'DBS - Finger': dbs_finger}

# Pair-wise scatter plots

In [ ]:
plt = sns.pairplot(lipid_data_df_without_na, y_vars=["Person", 'Blood Type', 'Date'], hue='Blood Type')
plt.savefig('../results/pair-plot.pdf')

# Dimension reduction

## PCA

In [ ]:
Path('../results/visualisation').mkdir(exist_ok=True)

In [ ]:
pca = PCA(n_components=2)
lipid_data_df_without_na_standardised = (lipid_data_df_without_na.drop(
    columns=['Person', 'Blood Type', 'Date'])-
                            lipid_data_df_without_na.drop(
    columns=['Person', 'Blood Type', 'Date']).mean())/lipid_data_df_without_na.drop(
    columns=['Person', 'Blood Type', 'Date']).std()
projected_lipid_data = pca.fit_transform(lipid_data_df_without_na_standardised)
print(pca.explained_variance_ratio_)
print(sum(pca.explained_variance_ratio_))

In [ ]:
pc_df = pd.DataFrame(data = projected_lipid_data , 
        columns = ['PC1', 'PC2'])
pc_df['Blood Type'] = lipid_data_df['Blood Type']
pc_df['Person'] = lipid_data_df['Person']

lmplot_persons = sns.lmplot( x="PC1", y="PC2",
  data=pc_df, 
  fit_reg=False, 
  hue='Person', # color by cluster
  legend=True,
  scatter_kws={"s": 50}) # specify the point size
lmplot_persons.savefig('../results/visualisation/persons_pca.pdf')

In [ ]:
lmplot_type = sns.lmplot( x="PC1", y="PC2",
  data=pc_df, 
  fit_reg=False, 
  hue='Blood Type', # color by cluster
  legend=True,
  scatter_kws={"s": 50}) # specify the point size
lmplot_type.savefig('../results/visualisation/type_pca.pdf')

## t-SNE

In [ ]:
tsne = TSNE(n_components=2, init='pca', learning_rate='auto')
projected_lipid_data = tsne.fit_transform(lipid_data_df_without_na_standardised)

In [ ]:
pc_df = pd.DataFrame(data = projected_lipid_data , 
        columns = ['TSNE1', 'TSNE2'])
pc_df['Blood Type'] = lipid_data_df['Blood Type']
pc_df['Person'] = lipid_data_df['Person']

lmplot_persons = sns.lmplot( x="TSNE1", y="TSNE2",
  data=pc_df, 
  fit_reg=False, 
  hue='Person', # color by cluster
  legend=True,
  scatter_kws={"s": 50}) # specify the point size
lmplot_persons.savefig('../results/visualisation/persons_tsne.pdf')

In [ ]:
lmplot_type = sns.lmplot( x="TSNE1", y="TSNE2",
  data=pc_df, 
  fit_reg=False, 
  hue='Blood Type', # color by cluster
  legend=True,
  scatter_kws={"s": 80}) # specify the point size
lmplot_type.savefig('visualisation/type_tsne.pdf')

# Statistical tests

In [ ]:
p_border = 0.01
Path('../results/statistic/friedman').mkdir(parents=True, exist_ok=True)
Path('../results/statistic/kruskal-wallis').mkdir(parents=True, exist_ok=True)

## Friedman test to test differences in lipid distributions over timepoints per blood type

In [ ]:
column = []
for key, value in blood_type_dict.items():
    counter = 0
    dates = value['Date'].unique()
    first_date_data = value[value['Date'] == dates[0]]
    second_date_data = value[value['Date'] == dates[1]]
    third_date_data = value[value['Date'] == dates[2]]
    for k in value.drop(columns=['Person', 'Blood Type', 'Date']).keys(): 
        _, pvalue = stats.friedmanchisquare(first_date_data[k], second_date_data[k], third_date_data[k])
        if pvalue <= p_border:
            counter += 1
    column.append(counter)

sns.heatmap(np.asarray(column)[:, np.newaxis],  linewidths=0.5, cmap='viridis',
            yticklabels=blood_type_dict.keys(), xticklabels=[], fmt='g', annot=True, cbar=False)
plt.savefig('statistic/friedman/friedmann_test_blood_type.pdf')

## Kruskal wallis test to test differences in lipid distributions between persons per blood type

In [ ]:
column = []
for key, value in blood_type_dict.items():
    counter = 0
    person = value['Person'].unique()
    first_person_data = value[value['Person'] == person[0]]
    second_person_data = value[value['Person'] == person[1]]
    third_person_data = value[value['Person'] == person[2]]
    fourth_person_data = value[value['Person'] == person[3]]
    
    for k in value.drop(columns=['Person', 'Blood Type', 'Date']).keys(): 
        _, pvalue = stats.kruskal(first_person_data[k], second_person_data[k],
                                            third_person_data[k], fourth_person_data[k])
        if pvalue <= p_border:
            counter += 1
    column.append(counter)

sns.heatmap(np.asarray(column)[:, np.newaxis],  linewidths=0.5, cmap='viridis',
            yticklabels=blood_type_dict.keys(), xticklabels=[], fmt='g', annot=True,
           cbar=False)
plt.savefig('../results/statistic/kruskal-wallis/kruskal-wallis_test_persons.pdf')

# Compare intergroup differences

## Shapiro-Wilk test for normality of lipid distributions

In [ ]:
for key, value in blood_type_dict.items():
    counter = 0
    column = []
    for k in value.drop(columns=['Person', 'Blood Type', 'Date']).keys():
        _, pvalue = shapiro(value[k])
        if pvalue <= p_border:
            counter += 1
    print(f'Number of rejected normal distributed variables: {counter} in {key}')

## Wilcoxon signed-rank test to test differences in lipid distribution between blood types. And visualise the number of differences with a heatmap.

In [ ]:
Path('statistic/wilcoxon').mkdir(parents=True, exist_ok=True)

In [ ]:
matrix = []
for first_key, first_value in blood_type_dict.items():
    column = []
    for second_key, second_value in blood_type_dict.items():
        counter = 0
        if first_key != second_key:
            for k in first_value.drop(columns=['Person', 'Blood Type', 'Date']).keys():
                statistics, pvalue = stats.wilcoxon(first_value[k], second_value[k], mode='exact')
                if pvalue <= p_border:
                    counter += 1
        column.append(counter)
    matrix.append(column)

mask = np.triu(matrix)
sns.heatmap(matrix, annot=True, linewidths=0.5, cmap='viridis',
            xticklabels=blood_type_dict.keys(), yticklabels=blood_type_dict.keys(), fmt='g', mask=mask)
plt.savefig('../results/statistic/wilcoxon/wilcoxon_signed_rank_test.pdf')

# Train a random forest classifier

In [ ]:
Path('../results/classification/multiclass').mkdir(parents=True, exist_ok=True)
Path('../results/classification/one_vs_rest').mkdir(parents=True, exist_ok=True)
Path('../results/classification/one_vs_one').mkdir(parents=True, exist_ok=True)

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 500, num = 5)]
max_depth = [int(x) for x in np.linspace(10, 100, num = 5)]
max_depth.append(None)
min_samples_split = [2, 3, 5]
min_samples_leaf = [1, 2, 4]
lw = 2
# Create the grid
param_grid = {'n_estimators': n_estimators,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
random_forest = RandomForestClassifier()
num_classes = len(lipid_data_df_without_na['Blood Type'].unique())
lipid_x = lipid_data_df_without_na.drop(columns=['Person', 'Blood Type', 'Date'])
label_encoder = LabelEncoder()
blood_type = label_encoder.fit_transform(lipid_data_df_without_na['Blood Type'])
logo = LeaveOneGroupOut()

## Multiclass

In [ ]:
all_true = []
all_prediction = []

auroc_list = []
outer_groups = lipid_data_df_without_na['Person']
list_shap_values = list()
list_test_sets = list()
list_permutation_imortance = list()
for train_index, test_index in logo.split(lipid_x, groups=outer_groups):
    train_val_data = lipid_x.loc[train_index]
    test_data = lipid_x.loc[test_index]
    blood_type_test_val = blood_type[train_index]
    blood_type_test = blood_type[test_index]
    inner_groups = outer_groups.loc[train_index]
    grid_search = GridSearchCV(estimator = random_forest, param_grid = param_grid, 
                          cv = logo, n_jobs = -1, verbose = 1)
    clf = grid_search.fit(train_val_data,  blood_type_test_val, groups = inner_groups)
    
    #explaining model
    explainer = shap.TreeExplainer(clf.best_estimator_)
    shap_values = explainer.shap_values(test_data)
    #for each iteration we save the test_set index and the shap_values
    list_shap_values.append(shap_values)
    list_test_sets.append(test_index)
    importance = permutation_importance(clf, test_data, blood_type_test)
    list_permutation_imortance.append(importance)
    
    # test stuff
    y_pred_proba = clf.predict_proba(test_data)
    y_pred = clf.predict(test_data)
    all_true.extend(blood_type_test)
    all_prediction.extend(y_pred_proba)
tmp = [permutation_imortance.importances_mean for permutation_imortance in list_permutation_imortance]
mean_importance = np.mean(tmp, axis=0)
tmp = [permutation_imortance.importances_std for permutation_imortance in list_permutation_imortance]
std_importance = np.mean(tmp, axis=0)

## Permutation importance

In [ ]:
forest_importances = pd.Series(mean_importance, index=lipid_x.keys())
forest_sd = pd.Series(std_importance, index=lipid_x.keys())
both = pd.concat([forest_importances, forest_sd], axis=1).sort_values(by=0, ascending=False)

ax = both[0][:10].plot.bar(yerr=both[1])
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean accuracy decrease")
plt.savefig('../results/classification/multiclass/permutation_feature_importance_multiclass.pdf')

## Shap diagram

In [ ]:
shap_values = np.concatenate(list_shap_values,axis=1)
class_int = 0
for i in range(len(label_encoder.classes_)):
    shap.summary_plot(shap_values[i], lipid_x, show=False)
    plt.savefig(f'../results/classification/multiclass/ \
                multiclass_{label_encoder.inverse_transform([i])[0]}_shap_beeswarm.pdf')
    plt.clf()

In [ ]:
shap.summary_plot(list(shap_values[:,:,:]), lipid_x, plot_type="bar",
                  class_names=label_encoder.classes_, show=False)
plt.savefig(f'../results/classification/multiclass/multiclass_shap_bar.pdf')

## Visualise AUROC

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(len(label_encoder.classes_)):
    type_true = np.array([1 if y == i else 0 for y in all_true])
    fpr[i], tpr[i], _ = roc_curve(type_true, np.array(all_prediction)[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])

#for i in range(len(label_encoder.classes_)):
#    plt.plot(fpr[i], tpr[i], lw=lw,
#        label=f"{label_encoder.inverse_transform([i])[0]} (area = {roc_auc[i]:0.2f})"
#)


all_fpr = np.unique(np.concatenate([fpr[i] for i in range(len(label_encoder.classes_))]))
mean_tpr = np.zeros_like(all_fpr)
for i in range(len(label_encoder.classes_)):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
mean_tpr /= len(label_encoder.classes_)
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle="-",
    linewidth=lw)

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.title('Multiclass')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.savefig('../results/classification/multiclass/multiclass_classifier.pdf')
plt.show()

## One vs. rest classification

In [ ]:
all_true = []
all_prediction = []
clf_list = []

outer_groups = lipid_data_df_without_na['Person']
list_shap_values = list()
list_test_sets = list()
mean_importance_list = list()
std_importance_list = list()

for one_type in range(len(label_encoder.classes_)):
    all_true_type = []
    all_prediction_group = []
    tmp_list_shap_values = list()
    tmp_list_test_sets = list()
    list_permutation_imortance = list()
    for train_index, test_index in logo.split(lipid_x, groups=outer_groups):
        ovr_blood_type = np.array([1 if y == one_type else 0 for y in blood_type])
        train_val_data = lipid_x.loc[train_index]
        test_data = lipid_x.loc[test_index]
        blood_type_test_val = ovr_blood_type[train_index]
        blood_type_test = ovr_blood_type[test_index]
        inner_groups = outer_groups.loc[train_index]
        grid_search = GridSearchCV(estimator = random_forest, param_grid = param_grid, 
                              cv = logo, n_jobs = -1, verbose = 1)
        clf = grid_search.fit(train_val_data, blood_type_test_val, groups = inner_groups)
        clf_list.append(clf)
        
        #explaining model
        explainer = shap.TreeExplainer(clf.best_estimator_)
        shap_values = explainer.shap_values(test_data)
        #for each iteration we save the test_set index and the shap_values
        tmp_list_shap_values.append(shap_values)
        tmp_list_test_sets.append(test_index)
        importance = permutation_importance(clf, test_data, blood_type_test)
        list_permutation_imortance.append(importance)
    
        # test stuff
        y_pred_proba = clf.predict_proba(test_data)
        y_pred = clf.predict(test_data)
        all_true_type.extend(blood_type_test)
        all_prediction_group.extend(y_pred_proba)
    
    list_shap_values.append(np.concatenate(tmp_list_shap_values,axis=1))
    list_test_sets.append(np.concatenate(tmp_list_test_sets, axis = 0))
    all_true.append(all_true_type)
    all_prediction.append(all_prediction_group)
    tmp = [permutation_imortance.importances_mean for permutation_imortance in list_permutation_imortance]
    mean_importance = np.mean(tmp, axis=0)
    tmp = [permutation_imortance.importances_std for permutation_imortance in list_permutation_imortance]
    std_importance = np.mean(tmp, axis=0)
    mean_importance_list.append(mean_importance)
    std_importance_list.append(std_importance)

## Permutation importance

In [ ]:
for i in range(len(label_encoder.classes_)):
    mean_importance = mean_importance_list[i]
    std_importance = std_importance_list[i]
    forest_importances = pd.Series(mean_importance, index=lipid_x.keys())
    forest_sd = pd.Series(std_importance, index=lipid_x.keys())
    both = pd. concat([forest_importances, forest_sd], axis=1).sort_values(by=0, ascending=False)

    fig, ax = plt.subplots()
    both[0][:10].plot.bar(yerr=both[1], ax=ax)
    ax.set_title("Feature importances using permutation on full model")
    ax.set_ylabel("Mean accuracy decrease")
    plt.savefig(f'../results/classification/one_vs_rest/ \
                {label_encoder.inverse_transform([i])[0]}_vs_rest_permutation_importance.pdf')
    plt.clf()

## SHAP

In [ ]:
for i in range(len(label_encoder.classes_)):
    shap_values = list_shap_values[i]
    test_set = list_test_sets[i]
    X = X_test = lipid_x.loc[test_set]
    shap.summary_plot(shap_values[1], X, show=False, max_display=10)
    plt.savefig(f'{label_encoder.inverse_transform([i])[0]}_vs_rest_shap_beeswarm.pdf')
    plt.clf()
    shap.summary_plot(list(shap_values[:,:,:]), X, plot_type="bar",
                  class_names=[label_encoder.inverse_transform([i])[0], 'Others'], show=False)
    plt.savefig(f'../results/classification/one_vs_rest/{label_encoder.inverse_transform([i])[0]}\
    _vs_rest_shap_bar.pdf')
    plt.clf()

## Visualise AUROC

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(len(label_encoder.classes_)):
    fpr[i], tpr[i], _ = roc_curve(all_true[i], np.array(all_prediction[i])[:, 1])
    roc_auc[i] = auc(fpr[i], tpr[i])

for i in range(len(label_encoder.classes_)):
    plt.plot(fpr[i], tpr[i], lw=lw,
        label=f"{label_encoder.inverse_transform([i])[0]} (area = {roc_auc[i]:0.2f})")


all_fpr = np.unique(np.concatenate([fpr[i] for i in range(len(label_encoder.classes_))]))
mean_tpr = np.zeros_like(all_fpr)
for i in range(len(label_encoder.classes_)):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
mean_tpr /= len(label_encoder.classes_)
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.title('One vs. Rest')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.savefig('../results/classification/one_vs_rest/one_vs_rest_classifier.pdf')
plt.show()

## One vs. one classification

In [ ]:
all_true = []
all_prediction = []

list_shap_values = list()
list_test_sets = list()
mean_importance_list = list()
std_importance_list = list()

for one_type in range(len(label_encoder.classes_)):
    all_true_type = []
    all_prediction_type = []

    for second_one_type in range(len(label_encoder.classes_)):
        if one_type == second_one_type:
            all_true_type.append([])
            all_prediction_type.append([])
            continue
        d = {one_type: 1, second_one_type: 0}
        ovo_blood_type = np.array([d[y]  for y in blood_type if y in d])
        indices = np.nonzero(np.isin(blood_type, [one_type, second_one_type]))
        outer_groups = lipid_data_df_without_na.loc[indices[0]]['Person']
        lipid_x = lipid_data_df_without_na.loc[indices[0]].drop(columns=['Person', 'Blood Type', 'Date'])
        all_prediction_ovo = []
        all_true_ovo = []
        for train_index, test_index in logo.split(lipid_x, groups=outer_groups):
            train_val_data = lipid_x.iloc[train_index]
            test_data = lipid_x.iloc[test_index]
            blood_type_test_val = ovo_blood_type[train_index]
            blood_type_test = ovo_blood_type[test_index]
            inner_groups = outer_groups.iloc[train_index]
            grid_search = GridSearchCV(estimator = random_forest, param_grid = param_grid, 
                                  cv = logo, n_jobs = -1, verbose = 1)
            clf = grid_search.fit(train_val_data, blood_type_test_val, groups = inner_groups)

            #explaining model
            explainer = shap.TreeExplainer(clf.best_estimator_)
            shap_values = explainer.shap_values(test_data)
            #for each iteration we save the test_set index and the shap_values
            tmp_list_shap_values.append(shap_values)
            tmp_list_test_sets.append(test_index)
            importance = permutation_importance(clf, test_data, blood_type_test)
            list_permutation_imortance.append(importance)
        
            # test stuff
            y_pred_proba = clf.predict_proba(test_data)
            y_pred = clf.predict(test_data)
            all_true_ovo.extend(blood_type_test)
            all_prediction_ovo.extend(y_pred_proba)
        all_true_type.append(all_true_ovo)
        all_prediction_type.append(all_prediction_ovo)
    all_true.append(all_true_type)
    all_prediction.append(all_prediction_type)

## Visualise AUROCS

In [ ]:
for one_type in range(len(label_encoder.classes_)):
    for second_one_type in range(len(label_encoder.classes_)):
        if one_type == second_one_type:
            continue
        all_true_ovo = all_true[one_type][second_one_type]
        all_prediction_ovo = all_prediction[one_type][second_one_type]
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        lw = 2


        fpr, tpr, _ = roc_curve(all_true_ovo, np.array(all_prediction_ovo)[:,1])
        roc_auc = auc(fpr, tpr)

        plt.plot(fpr, tpr, lw=lw,
            label=f"{label_encoder.inverse_transform([one_type])[0]}"\
                 f" vs. {label_encoder.inverse_transform([second_one_type])[0]} (area = {roc_auc:0.2f})")

        plt.plot([0, 1], [0, 1], "k--", lw=lw)
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.title(f'{label_encoder.inverse_transform([one_type])[0]} vs. One')
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.legend(loc="lower right")
    plt.savefig(f'../results/classification/one_vs_one/ \
    {label_encoder.inverse_transform([one_type])[0]}_vs_one_classifier.pdf')
    plt.show()